# Example: paired-end sequencing

This is an example generated from this source
file: [`pe-example.jl`](https://github.com/bioinfologics/Pseudoseq.jl/blob/masterexamples/sequencing/pe-example.jl)
You are seeing the
jupyter notebook version. The corresponding online documentation page can
be found here: [`pe-example.html`](https://bioinfologics.github.io/Pseudoseq.jl/latest/examples/sequencing/pe-example),
and the script can be found here: [`pe-example.jl`](./pe-example.jl)

For the simulation we are going to:

1. Create a pool of 5000 copies of a reference genome.
2. Fragment the DNA molecules in the pool, to an average length of 700bp.
3. Subsample the molecules in the pool to achieve approximatly 50x coverage.
4. Create a set of 250bp paired-end reads.
5. Apply errors to the paired-end reads at a rate of 0.001 (.1%).
6. Generate an output FASTQ file.

In [1]:
using Pseudoseq.Sequencing

## Using the `sequence` method

First, let's see how we do this with the `sequence` method.
The first two parameters we give to the function will be the input genome we
want to sequence, and the destination FASTQ file for output reads.
Here we are setting:
- The number of genome copies in the molecule pool to 5000.
- The average fragment size to 700bp.
- The sampling coverage to 50x.
- The read length to 250bp.
- The per base read error rate to 0.001.
- The fact we want paired-ends of fragments to be read (`paired`) to true.

In [2]:
sequence("ecoli-ref.fasta", "pe-reads.fastq"; ng = 5000, flen = 700, cov = 50, paired = true, rdlen = 250, err = 0.001)

┌ Info: - ✔ Created pool of 5000 copies of a 4639675bp genome
└ @ Pseudoseq.Sequencing /home/runner/.julia/packages/Pseudoseq/18MzR/src/sequencing/sequence.jl:45
┌ Info: - ✔ Created pool of fragments with an average length of 700bp
└ @ Pseudoseq.Sequencing /home/runner/.julia/packages/Pseudoseq/18MzR/src/sequencing/sequence.jl:55
┌ Info: - ✔ Subsampled pool at 50X coverage (463967 molecules)
└ @ Pseudoseq.Sequencing /home/runner/.julia/packages/Pseudoseq/18MzR/src/sequencing/sequence.jl:62
┌ Info: - ✔ Created set of 250bp paired-end reads
└ @ Pseudoseq.Sequencing /home/runner/.julia/packages/Pseudoseq/18MzR/src/sequencing/sequence.jl:66
┌ Info: - ✔ Applied sequencing errors at a per-base rate of 0.001
└ @ Pseudoseq.Sequencing /home/runner/.julia/packages/Pseudoseq/18MzR/src/sequencing/sequence.jl:73
┌ Info: - ✔ Wrote 650408 paired end reads to FASTQ file
└ @ Pseudoseq.Sequencing /home/runner/.julia/packages/Pseudoseq/18MzR/src/sequencing/Reads.jl:212


## Using the `Molecules` tranformation methods.

Here's how to achieve the same thing, using the Pseudoseq API. It is nessecery to
use the API if you want to do something that is not anticipated by the available
functionality of the `sequence` method: the cost of conveinience is fewer options.

Starting with a FASTA formatted file containing the genome we want to sequence,
we create a pool with 5000 copies of the genome.

In [3]:
pool = Molecules("ecoli-ref.fasta", 5000)

Pool of 5000 DNA molecules:
 All molecules are of the same size: 4639675


Next we use the fragment function to make a pool of shorter DNA molecules.

In [4]:
cutpool = fragment(pool, 700)

Pool of 33139993 DNA molecules:
 Maximum molecule size: 12077
 Average molecule size: 700
 Minimum molecule size: 1


We need to determine the number of molecules to sample, and subsample the pool:

In [5]:
genome_size = 4639675
expected_coverage = 50
read_length = 250

N = needed_sample_size(expected_coverage, genome_size, read_length)
N = div(N, 2) # Divide by 2 as we're doing paired end sequencing.

sampledpool = subsample(cutpool, N)

Pool of 463967 DNA molecules:
 Maximum molecule size: 8725
 Average molecule size: 701
 Minimum molecule size: 1


We now want to create a set of paired-end reads. We want our reads to be 250bp
in length.

In [6]:
pe_reads = paired_reads(sampledpool, 250)

Now we have some reads, we should mark positions in the reads that are destined
to be errors in the output FASTQ.

In [7]:
pe_w_errs = mark_errors(pe_reads, 0.001)

Now we have some paired end reads and have marked some positions as errors, we
can generate FASTQ files.

In [8]:
generate("pe-reads.fastq", pe_w_errs)

┌ Info: - ✔ Wrote 648742 paired end reads to FASTQ file
└ @ Pseudoseq.Sequencing /home/runner/.julia/packages/Pseudoseq/18MzR/src/sequencing/Reads.jl:212


## Constructing a pipeline of `Processors`.

As a convenience, some users may prefer to use pipelines of `Processors`
These behave like curried versions of the `Molecules` transformation methods.
First let's define our starting `Molecules` pool:

In [9]:
pool = Molecules("ecoli-ref.fasta", 5000)

Pool of 5000 DNA molecules:
 All molecules are of the same size: 4639675


To make a Processor, use a `Molecules` transformation method, but do not
provide a `Molecules` value as a first argument. So let's make Processors for
each step of our paired end sequencing pipeline.

In [10]:
cutter = fragment(700)
sampler = subsample(N) # Remember how to computed N previously.

Pseudoseq.Sequencing.SubSampler(463967)

Next we can construct the pipeline using standard julia function pipelining syntax:

In [11]:
pool |> cutter |> sampler

Pool of 463967 DNA molecules:
 Maximum molecule size: 8970
 Average molecule size: 699
 Minimum molecule size: 1


You can also compose the processors together into one whole function.

\circ.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*